# Building an HMMlogo
## 1 step - Retrieve all the sequences of the structures that present the Kunitz domain. 
How? it is possible to do so by doing a [PDB advance search](https://www.rcsb.org/search/advanced) with the query:

```(Annotation Identifier = "PF00014" AND Annotation Type = "Pfam") AND Resolution (Å) = [0 - 3]```

we retrieved ```236``` structures ID

then we download the PDB IDs in ```.cvs``` format and we run the command to see if every sequence has been taken. 

In [2]:
!wc -l rcsb_pdb_pdb_ids_20200423153832.csv

236 rcsb_pdb_pdb_ids_20200423153832.csv


to retrieve the fasta sequences we will perform a pairwise alignment between our 236 structure and a well annotated kunitz-domain structure ```3TGI:I``` to retrieve those specific chains that do actually contain the kunitz domain. We will perform the alignments on [PDBeFold](https://www.ebi.ac.uk/msd-srv/ssm/cgi-bin/ssmserver). Here we require a certain file format. In this case, it would require a list of pdb id w/o quotes. We will get rid of them with ```sed``` command: 

In [11]:
!sed 's,",,g' rcsb_pdb_pdb_ids_20200423153832.csv > rcsb_pdb_pdb_ids_20200423153832.txt 
!head rcsb_pdb_pdb_ids_20200423153832.txt

3OTJ
3D65
3DXC
3DXD
3DXE
2RA3
2TPI
2TGP
5PTI
5TPT


Then we upload the file in the input page of PDBeFold and we'll set this parameters:
```Lowest acceptable match (%) = 80%``` for each input. 

Matches found were ```258```. Now we can ```Download target sequences``` in fasta format. 

I saved it as ```seq_kunitz.fasta```

In [20]:
!grep ">" seq_kunitz.fasta | wc -l 

258


## step 2 - clustering our results

Now that we have 258 sequences we want to cluster them in order to have representative sequences to do the multiple sequences alignment. 
We'll do:

In [1]:
!cd-hit -i seq_kunitz.fasta -o cluster.fasta

Program: CD-HIT, V4.8.1 (+OpenMP), Oct 26 2019, 14:51:47
Command: cd-hit -i seq_kunitz.fasta -o cluster.fasta

Started: Thu Apr 23 18:08:36 2020
                            Output                              
----------------------------------------------------------------
total seq: 258
longest and shortest : 66 and 41
Total letters: 14655
Sequences have been sorted

Approximated minimal memory consumption:
Sequence        : 0M
Buffer          : 1 X 10M = 10M
Table           : 1 X 65M = 65M
Miscellaneous   : 0M
Total           : 75M

Table limit with the given memory limit:
Max number of representatives: 4000000
Max number of word counting entries: 90511720

comparing sequences from          0  to        258

      258  finished         20  clusters

Approximated maximum memory consumption: 75M
writing new database
writing clustering information
program completed !

Total CPU time 0.10


In [12]:
!head cluster.fasta

>PDB:1yld:B TRYPSIN/BPTI COMPLEX MUTANT
RPDFXLEPPYTGPCKARIIRYFYNAPDGLXQTFVYGGCRAKRNNFKSAEDXMRTXG
>PDB:1fak:I HUMAN TISSUE FACTOR COMPLEXED WITH COAGULATION FAC
APDFCLEPPYDGPCRALHLRYFYNAKAGLCQTFYYGGCLAKRNNFESAEDCMRTC
>PDB:5jb7:B A SIMPLIFIED BPTI VARIANT CONTAINING 24 ALANINES O
RPAFCLEAPYAGPGAAAIIRYFYNAAAGAAQAFVYGGVAAKRNNFASAAAALAACAAA
>PDB:1t8l:D CRYSTAL STRUCTURE OF THE P1 MET BPTI MUTANT- BOVIN
MRPDFCLEPPYTGPCMARIIRYFYNAKAGLCQTFVYGGCRAKRNNFKSAEDCLRTCGGA
>PDB:5zj3:B TEXTILININ-1, A KUNITZ-TYPE SERINE PROTEASE INHIBI
KDRPDFCELPADTGPCRVRFPSFYYNPDEKKCLEFIYGGCEGNANNFITKEECESTCA


Number of clusters is:

In [9]:
!grep '>' cluster.fasta| wc -l 

20


Let's inspect our result by extracting the list of ids first:

In [5]:
!cat cluster.fasta|grep ">"|awk 'BEGIN {FS=OFS=":"} {print $2, $3}'| cut -d " " -f 1 > curated_seed.list

In [9]:
!grep '' cluster.fasta

>PDB:1yld:B TRYPSIN/BPTI COMPLEX MUTANT
>PDB:1t8l:D CRYSTAL STRUCTURE OF THE P1 MET BPTI MUTANT- BOVIN


In [10]:
!head curated_seed.list

1yld:B
1fak:I
5jb7:B
1t8l:D
5zj3:B
4u32:X
4dtg:K
6q61:A
1zjd:B
5yv7:A


```1yld:B ---> mutant
1fak:I ---> mutant
5jb7:B ---> mutant
1t8l:D ---> mutant
5zj3:B ---> ok
4u32:X ---> ok
4dtg:K ---> ok
6q61:A ---> ok
1zjd:B ---> ok wt
5yv7:A ---> seems ok
1dtx:A ---> ok
1kth:A ---> ??
4bqd:B ---> ok
3m7q:B ---> ok
1zr0:B ---> ok
4u30:X ---> ok
4ntx:B ---> ok
1yc0:I ---> ok
1bun:B ---> ok
1d0d:A ---> RMSD value very high, low seq identity, i will discard it```

let's delete those clusters that are not useful:

In [14]:
!tail +5 curated_seed.list | head -15 > hmm_seq_ids.list
!wc hmm_seq_ids.list

 15  15 105 hmm_seq_ids.list


Once the fasta format file is filtered out with our alignments we can perform the hmmbuild

## step 3 - building the hmm

In order to build the hmm model, we need to perform multiple sequences alignment, we will do it on [PDBeFold tool](https://www.ebi.ac.uk/msd-srv/ssm/cgi-bin/ssmserver). 

Our input file would be ```hmm_seq_ids.list```. 

we will download the alignment from the result page as ```multi_alicluster.seq```

In [16]:
!head mult_alicluster.seq

>PDB:5zj3:B TEXTILININ-1, A KUNITZ-TYPE SERINE PROTEASE INHIBI
-----kdrPd-fC-ELPADT--GP-CRvRFPSFYYNPDEKKCLEFIYGgCEG-NANNFITKEECESTCa-
--

>PDB:4u32:X HUMAN MESOTRYPSIN COMPLEXED WITH HAI-2 KUNITZ DOMA
--------Hd-fC-LVSKVV--GR-CRaSMPRWWYNVTDGSCQLFVYGgCDG-NSNNYLTKEECLKKC--
--

>PDB:4dtg:K HEMOSTATIC EFFECT OF A MONOCLONAL ANTIBODY MAB 202
------ekPd-fC-FLEEDP--GI-CRgYITRYFYNNQTKQCERFKYGgCLG-NMNNFETLEECKNICed


Now, we can build our **Hidden markov model** .

In [17]:
!hmmbuild -n kunitz kunitz_newlogo.hmm mult_alicluster.seq 

# hmmbuild :: profile HMM construction from multiple sequence alignments
# HMMER 3.3 (Nov 2019); http://hmmer.org/
# Copyright (C) 2019 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# input alignment file:             mult_alicluster.seq
# output HMM file:                  kunitz_newlogo.hmm
# name (the single) HMM:            kunitz
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

# idx name                  nseq  alen  mlen eff_nseq re/pos description
#---- -------------------- ----- ----- ----- -------- ------ -----------
1     kunitz                  15    72    60     1.92  0.931 

# CPU time: 0.08u 0.00s 00:00:00.08 Elapsed: 00:00:00.09


Results are in two files, I can generate a HMMlogo by going on https://skylign.org/

The logo will be like this:
![hmm_logo](../Images/hmmlogo.png)

If we compare it with pfam hmm logo we'll see that most of the conserved residues are present:
![pfamlogo](../Images/WT_hmmlogo.png)


